<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive-into-Deep-Learning-Notes-/blob/main/11_2_attention_pooling_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install d2l==1.0.0-alpha1.post0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.0 MB/s eta 0:00:00


## 11.2 Attention Pooling by Similarity

* `torch.sort(input)`: Sorts the elements of the `input` tensor along a given dimension in an **ascending order** by value. 

In [2]:
import torch

X = torch.tensor([[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10, 11, 12],
                  [13, 14, 15, 16]])
sorted, indices = torch.sort(X, descending=True)
sorted, indices

(tensor([[ 4,  3,  2,  1],
         [ 8,  7,  6,  5],
         [12, 11, 10,  9],
         [16, 15, 14, 13]]), tensor([[3, 2, 1, 0],
         [3, 2, 1, 0],
         [3, 2, 1, 0],
         [3, 2, 1, 0]]))

* `torch.rand(size)`: Returns a tensor filled with random numbers from a uniform distribution on the interval $[0, 1)$.

In [3]:
torch.rand(2)

tensor([0.6401, 0.0659])

* `torch.randn(size)`: Returns a tensor filled with random numbers from a normal distribution with mean 0 and variance (also called the standard normal distribution).

In [4]:
torch.randn(2)

tensor([-0.2631,  0.8598])

In [5]:
n = 4
x_train, _ = torch.sort(torch.rand(n) * 5)
x_val = torch.arange(0, 4, 1)

In [6]:
x_train = x_train.reshape((-1, 1))
x_val = x_val.reshape((1, -1))
x_train, x_val

(tensor([[2.0687],
         [2.3106],
         [2.4466],
         [3.9010]]), tensor([[0, 1, 2, 3]]))

In [7]:
dist = x_train - x_val    # Broadcasting!!
dist

tensor([[ 2.0687,  1.0687,  0.0687, -0.9313],
        [ 2.3106,  1.3106,  0.3106, -0.6894],
        [ 2.4466,  1.4466,  0.4466, -0.5534],
        [ 3.9010,  2.9010,  1.9010,  0.9010]])

In [11]:
x = torch.tensor([[1, 0.2],
                  [3, 0.4]])

Y = torch.abs(x) < 1         # False = 0 and True = 1
Y

tensor([[False,  True],
        [False,  True]])